In [2]:
import pandas as pd
import numpy as np

# Read in our data, and fill missing values
data = pd.read_csv("clean_weather.csv", index_col=0)
data = data.ffill()

# Display a sequence of temperatures
data["tmax"].head(10)

1970-01-01    60.0
1970-01-02    52.0
1970-01-03    52.0
1970-01-04    53.0
1970-01-05    52.0
1970-01-06    50.0
1970-01-07    52.0
1970-01-08    56.0
1970-01-09    54.0
1970-01-10    57.0
Name: tmax, dtype: float64

Training Set: The data that the model learns from. The model's weights and parameters are adjusted based on this set.
Validation Set: Used to evaluate the model during training for tuning hyperparameters and preventing overfitting. The model does not learn from this data but is evaluated on it periodically.
Test Set: A final set used to evaluate the model's performance after all training and tuning are complete. The test set is meant to represent how the model will perform on truly unseen data.

In [3]:
from sklearn.preprocessing import StandardScaler
import math

# Define predictors and target
PREDICTORS = ["tmax", "tmin", "rain"]
TARGET = "tmax_tomorrow"

# Scale our data to have mean 0
scaler = StandardScaler()
data[PREDICTORS] = scaler.fit_transform(data[PREDICTORS])

# Split into train, valid, test sets
np.random.seed(0)
split_data = np.split(data, [int(.7*len(data)), int(.85*len(data))])
(train_x, train_y), (valid_x, valid_y), (test_x, test_y) = [[d[PREDICTORS].to_numpy(), d[[TARGET]].to_numpy()] for d in split_data]

In [4]:
def init_params(layer_conf):
    layers = []
    for i in range(1, len(layer_conf)):
        np.random.seed(0)
        k = 1/math.sqrt(layer_conf[i]["hidden"])
        #k is a scaling factor based on the size of the hidden layer. This scaling factor ensures that the weights are initialized within a range that prevents exploding or vanishing gradients.
        i_weight = np.random.rand(layer_conf[i-1]["units"], layer_conf[i]["hidden"]) * 2 * k - k
        #i_weight represents the weights connecting the inputs to the hidden units in the current layer

        h_weight = np.random.rand(layer_conf[i]["hidden"], layer_conf[i]["hidden"]) * 2 * k - k
        #h_weight represents the recurrent weights connecting the hidden units at the current time step to the hidden units at the next time step.
        h_bias = np.random.rand(1, layer_conf[i]["hidden"]) * 2 * k - k

        o_weight = np.random.rand(layer_conf[i]["hidden"], layer_conf[i]["output"]) * 2 * k - k
        o_bias = np.random.rand(1, layer_conf[i]["output"]) * 2 * k - k
        #o_weight represents the weights connecting the hidden units to the output units.
        #This matrix has shape (hidden, output) and is initialized with values in the range [-k,k]

        layers.append(
            [i_weight, h_weight, h_bias, o_weight, o_bias]
        )
    return layers

In [5]:
def forward(x, layers):
    hiddens = []
    outputs = []
    for i in range(len(layers)):
        i_weight, h_weight, h_bias, o_weight, o_bias = layers[i]
        hidden = np.zeros((x.shape[0], i_weight.shape[1]))
        output = np.zeros((x.shape[0], o_weight.shape[1]))
        #after unpacking parameters, we loop through each time step j in the sequence
        for j in range(x.shape[0]):
            input_x = x[j,:][np.newaxis,:] @ i_weight#input at time j is projected into hidden space
            hidden_x = input_x + hidden[max(j-1,0),:][np.newaxis,:] @ h_weight + h_bias
            #retrieve hidden state from previous step and project it into hidden space h_weight
            # Activation.  tanh avoids outputs getting larger and larger.
            hidden_x = np.tanh(hidden_x)
            # Store hidden for use in backprop
            hidden[j,:] = hidden_x

            # Output layer
            output_x = hidden_x @ o_weight + o_bias
            #calc by projecting the hidden state into the output space using output weight matrix
            output[j,:] = output_x
        hiddens.append(hidden)
        outputs.append(output)
    return hiddens, outputs[-1]

Calculate and update the gradients of the weights and biases based on the loss gradienets received from forward pass. Backpropagation through time

In [7]:
def backward(layers, x, lr, grad, hiddens):
    #layers: list of initialized parameters for each layer as returned by init parametsts
    # lr; learning rate
    # grad: grad of the loss wrt output at each time step
    #hidden states computed during fw
    for i in range(len(layers)):
        i_weight, h_weight, h_bias, o_weight, o_bias = layers[i]
        hidden = hiddens[i]
        next_h_grad = None
        i_weight_grad, h_weight_grad, h_bias_grad, o_weight_grad, o_bias_grad = [0] * 5

        for j in range(x.shape[0] - 1, -1, -1):#loop over time steps backward
            # Add newaxis in the first dimension
            out_grad = grad[j,:][np.newaxis, :]

            # Output updates
            # np.newaxis creates a size 1 axis, in this case transposing matrix
            o_weight_grad += hidden[j,:][:, np.newaxis] @ out_grad
            o_bias_grad += out_grad

            # Propagate gradient to hidden unit
            #This line calculates how much the loss at the output affects the hidden state. It propagates the gradient back from the output to the hidden layer using the output weight (o_weight).
            h_grad = out_grad @ o_weight.T
            
            #add gradient from the next hidden state if we re not at the last time step, capturing influence of future time steps on current hiddens state
            if j < x.shape[0] - 1:
                # Then we multiply the gradient by the hidden weights to pull gradient from next hidden state to current hidden state
                hh_grad = next_h_grad @ h_weight.T
                # Add the gradients together to combine output contribution and hidden contribution
                h_grad += hh_grad

            # Pull the gradient across the current hidden nonlinearity
            # derivative of tanh is 1 - tanh(x) ** 2
            # since the hidden state uses tanh as activation, we apply yhis derivative h_grad to account for nonlinearity
            tanh_deriv = 1 - hidden[j][np.newaxis,:] ** 2

            # next_h_grad @ np.diag(tanh_deriv_next) multiplies each element of next_h_grad by the deriv
            # Effect is to pull value across nonlinearity
            h_grad = np.multiply(h_grad, tanh_deriv)

            # Store to compute h grad for previous sequence position
            next_h_grad = h_grad.copy()

            # If we're not at the very beginning
            #we calc the grad of h_weight by taking th e gradient h_grad and multiplying it by the hidden state from the previous time step
            
            if j > 0:
                # Multiply input from previous layer by post-nonlinearity grad at current layer
                h_weight_grad += hidden[j-1][:, np.newaxis] @ h_grad
                h_bias_grad += h_grad

            i_weight_grad += x[j,:][:,np.newaxis] @ h_grad
            #calc gradient of input to hidden weights by multiplying the input at the current time step w h_grad

        # Normalize lr by number of sequence elements
        #update weights
        lr = lr / x.shape[0]
        i_weight -= i_weight_grad * lr
        h_weight -= h_weight_grad * lr
        h_bias -= h_bias_grad * lr
        o_weight -= o_weight_grad * lr
        o_bias -= o_bias_grad * lr
        layers[i] = [i_weight, h_weight, h_bias, o_weight, o_bias]
    return layers

In [9]:
def mse(actual, predicted):
    return np.mean((actual-predicted)**2)

def mse_grad(actual, predicted):
    return (predicted - actual)
#compute the gradient of the mse loss wrt predictions

Training rnn:

In [10]:
epochs = 250#no of time the entire training dataset is passed through the rnn during training
lr = 1e-5

layer_conf = [
    {"type":"input", "units": 3},#in this example the input layer has 3 features :tmix, tmax, rain
    {"type": "rnn", "hidden": 4, "output": 1}# the rnn layer has 4 hidden units and outputs a single value per time step
]
layers = init_params(layer_conf)

for epoch in range(epochs):
    sequence_len = 7
    epoch_loss = 0
    for j in range(train_x.shape[0] - sequence_len):#inner loop over training data
        #training data sliced into overlapping sequences: a 7 time step seq of predcitors and a corresp t time step seq of targets
        seq_x = train_x[j:(j+sequence_len),]
        seq_y = train_y[j:(j+sequence_len),]
        #forward pass by computing predictions and hidden states
        hiddens, outputs = forward(seq_x, layers)
        #calculate grad and perform backward pass
        grad = mse_grad(seq_y, outputs)
        params = backward(layers, seq_x, lr, grad, hiddens)
        #update training loss for the epoch
        epoch_loss += mse(seq_y, outputs)

    #every 50 epochs we have fw on validared data
    if epoch % 50 == 0:
        sequence_len = 7
        valid_loss = 0
        for j in range(valid_x.shape[0] - sequence_len):
            seq_x = valid_x[j:(j+sequence_len),]
            seq_y = valid_y[j:(j+sequence_len),]
            _, outputs = forward(seq_x, layers)
            valid_loss += mse(seq_y, outputs)

        print(f"Epoch: {epoch} train loss {epoch_loss / len(train_x)} valid loss {valid_loss / len(valid_x)}")

Epoch: 0 train loss 3122.594400144508 valid loss 2171.318686210202
Epoch: 50 train loss 30.593193275313553 valid loss 30.568271740103356
Epoch: 100 train loss 25.263986813543724 valid loss 24.435517510355645
Epoch: 150 train loss 22.956762429531295 valid loss 22.177010971976845
Epoch: 200 train loss 22.3067743277042 valid loss 21.557992202834157
